##  [망고플레이트 크롤링](https://www.mangoplate.com/)    
- **가게 1개씩 크롤링**   
- 참고 사이트 : https://juwon2021.tistory.com/414

### ** 시작 전 확인 사항 **
  Window terminal에서   
- conda activate   
- conda activate ml-env   
  (ml-env) 환경 확인 후   
- jupyter notebook   
- conda install -c anaconda requests   
- conda install bs4   
- conda install lxml   
- conda install -c conda-forge selenium   
- conda install tqdm   
- connda install -c conda-forge folium   
- connda install -c conda-forge googlemaps   
- pip install chromedriver-autoinstaller   

### 가게 1개씩 크롤링

In [1]:
# 필요한 라이브러리를 임포트하기

# 시스템
import sys
import os

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 지역 키워드를 입력하기
keyword = '강남'

# page 정하기
page = 1

In [3]:
# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
driver.maximize_window() # 창 최대화

# 키워드가 입력된 망고플레이트 사이트에 들어가기
driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)) 
time.sleep(2)

In [4]:
# 첫 번째 음식점 클릭
element = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child(1) > div:nth-child(1) > figure > a > div > img'
driver.find_element(By.CSS_SELECTOR, element).click()

In [5]:
# 가게 이름 크롤링
element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
title_raw = driver.find_element(By.CSS_SELECTOR, element)
title = title_raw.text
print(title, type(title))

미라이 <class 'str'>


In [6]:
# 가게 전체 평점 점수 크롤링
element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
total_raw = driver.find_element(By.CSS_SELECTOR, element)
total = total_raw.text
print(total, type(total))

4.8 <class 'str'>


In [7]:
# 조회수 크롤링
try:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
    view_raw = driver.find_element(By.CSS_SELECTOR, element)
    view = view_raw.text
except:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
    view_raw = driver.find_element(By.CSS_SELECTOR, element)
    view = view_raw.text
    
print(view, type(view))


510,329 <class 'str'>


In [8]:
# 리뷰 개수 크롤링
try:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
    review_raw = driver.find_element(By.CSS_SELECTOR, element)
    num_review = review_raw.text

except:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
    review_raw = driver.find_element(By.CSS_SELECTOR, element)
    num_review = review_raw.text

print(num_review, type(num_review))

127 <class 'str'>


In [9]:
# 별표 개수 크롤링
try:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
    star_raw = driver.find_element(By.CSS_SELECTOR, element)
    num_star = star_raw.text

except:
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
    star_raw = driver.find_element(By.CSS_SELECTOR, element)
    num_star = star_raw.text

print(num_star, type(num_star))

10,337 <class 'str'>


In [10]:
# 음식 종류 및 메뉴 크롤링
element1 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(3) > td > span'
type_raw = driver.find_element(By.CSS_SELECTOR, element1)
type_raw = type_raw.text
food_type = type_raw

try:
    element2 = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(9) > td > ul'
    menu_raw = driver.find_element(By.CSS_SELECTOR, element2)
    food_menu = menu_raw.text
    food_type = [type_raw, food_menu]
except:
    pass

print(food_type)

['이자카야 / 오뎅 / 꼬치', '사시미8종 (2인)\n41,000원\n마구로치즈아에\n19,000원\n양갈비 스미비 야끼\n23,000원\n노도구로 시오 야끼\n35,000원\n규 스테이크\n41,000원']


In [11]:
# 데이터 프레임 만들기
a = {'Title' : title,
     'Point' : total,
     'View' : view,
     'Review' : num_review,
     'Star' : num_star,
     'Type' : food_type}

df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()
df.head()

,Title,Point,View,Review,Star,Type
0,미라이,4.8,"510,329",127,"10,337","[이자카야 / 오뎅 / 꼬치, 사시미8종 (2인)\n41,000원\n마구로치즈아에\..."
